#### Imports defined

In [1]:
import altair as alt ## v 4.0.0 required.
import pandas as pd

#### Reading csv file and storing it into a DataFrame.

In [2]:
df = pd.read_csv('data/esb_tidy.csv')

In [3]:
df.head()

,Council Name,Council Type,Year,Month,ESB Connection
0,Cork,City Council,2006,Jan,39
1,Cork,City Council,2006,Feb,22
2,Cork,City Council,2006,Mar,93
3,Cork,City Council,2006,Apr,31
4,Cork,City Council,2006,May,162


#### Replacing the column whitespaces with "_" in the DataFrame.

In [4]:
df.columns = [i.replace(' ','_') for i in df.columns]  # Column names with whitespace resulted in Javascript Error
                                                       # while generating plots.

#### Function to format a DataFrame.

In [5]:
def untidy_df(df):
    col_length = df.shape[0]
    start = end = 0
    rows = []
    
    while (col_length > 0) :

        sliced_df = df.loc[start:(end+11)]
        df.drop(sliced_df.index,inplace=True)
        df.reset_index(drop=True, inplace=True)

        Council_Name = list(sliced_df['Council_Name'].unique())[0]
        Council_Type = list(sliced_df['Council_Type'].unique())[0]
        Year = list(sliced_df['Year'].unique())[0]

        ESB_Connections = list(sliced_df['ESB_Connection'].values)

        row = [Council_Name,Council_Type,Year]
        
        row.extend(ESB_Connections)
        rows.append(row)
        
        col_length = df.shape[0]
        
    df_untidy = pd.DataFrame(columns = ['Council_Name','Council_Type','Year','Jan','Feb','Mar','Apr','May',
                                        'Jun','Jul','Aug','Sept','Oct','Nov','Dec'], data = rows)
    
    return df_untidy
    

#### Chart 1 : City Council ESB Connections over the Years.

In [6]:
chart1 = df.copy()
chart1['Year'] = chart1['Year'].astype(str)
chart1['Year'] = pd.to_datetime(chart1['Year'] + ' ' + chart1['Month'])

In [7]:
highlight = alt.selection(type='single', on='mouseover',
                          fields=['Council_Name'], nearest=True)

base = alt.Chart(chart1).encode(
    x = 'Year:T',
    y = 'ESB_Connection:Q',
    color = 'Council_Name:N'
).transform_filter(alt.datum['Council_Type'] == 'City Council')

points = base.mark_circle().encode(
    opacity = alt.value(3)
).add_selection(
    highlight
).properties(
    width = 800,
    title = 'City Council ESB Connection over the Years'
)

lines = base.mark_line().encode(
    size = alt.condition(~highlight, alt.value(1), alt.value(3))
)


(points + lines).save('C00246376_Python_Assignment_3_Part_3/static/chart1.html')

points + lines

alt.LayerChart(...)

#### Chart 2 : Yearly ESB Connections by Council Type.

In [8]:
chart2 = df.copy()
chart2 = untidy_df(chart2)
chart2['Year'] = chart2['Year'].astype(str)

chart2 = chart2.groupby(['Year','Council_Type'],as_index=False).agg(
    {
        'Jan' :'sum',
        'Feb' :'sum',
        'Mar' :'sum',
        'Apr' :'sum',
        'May' :'sum',
        'Jun' :'sum',
        'Jul' :'sum',
        'Aug' :'sum',
        'Sept':'sum',
        'Oct' :'sum',
        'Nov' :'sum',
        'Dec' :'sum',
    }
    )

chart2['ESB_Connection'] = chart2.sum(axis=1)

In [9]:
chart = alt.Chart(chart2)

colour = alt.Scale(domain = ('City Council', 'County Council'),
                      range = ['grey', 'orange'])

bars = chart.mark_bar().encode(
    x = alt.X('Council_Type:N', title = None),
    y = alt.Y('ESB_Connection:Q'),
    color = alt.Color('Council_Type:N', scale = colour),
    column = 'Year:O'
).properties(
    width = 50,
    title = 'Yearly ESB Connections by Council Type',
    background = 'white'
).configure_facet(
    spacing = 20
)

bars.save('C00246376_Python_Assignment_3_Part_3/static/chart2.html')

bars


alt.Chart(...)

#### Chart 3 : Yearly ESB Connection as a Percentage of Total.

In [10]:
chart3 = df.groupby(['Year'],as_index=False).agg(
    {'ESB_Connection' :'sum'}
)

In [11]:
bars = alt.Chart(chart3).transform_joinaggregate(
    TotalESB = 'sum(ESB_Connection)',
).transform_calculate(
    PercentOfTotal = "datum.ESB_Connection / datum.TotalESB"
).mark_bar().encode(
    alt.X('PercentOfTotal:Q', axis=alt.Axis(format = '.0%')),
    y = 'Year:N',
)

text = bars.mark_text(
    align = 'left',
    baseline = 'middle',
    dx = 3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text = 'ESB_Connection'
)

(bars + text).properties(
    title = 'Yearly ESB Connection as a Percentage of Total',
    height = 300,
).save('C00246376_Python_Assignment_3_Part_3/static/chart3.html')

bars + text

alt.LayerChart(...)

#### Chart 4 : City Wise ESB Conections over the Years.

In [12]:
chart4 = df.groupby(['Year','Council_Name','Council_Type'],as_index=False).agg(
    {'ESB_Connection' :'sum'}
)

chart4 = chart4[chart4.Council_Type == 'City Council']

In [13]:
bars = alt.Chart(chart4).mark_bar(size=25).encode(
    x=alt.X('sum(ESB_Connection):Q', stack='zero'),
    y=alt.Y('Year:Q'),
    color=alt.Color('Council_Name')
).properties(
    width = 700,
    title = 'City Wise ESB Conections over the Years'
)

bars.save('C00246376_Python_Assignment_3_Part_3/static/chart4.html')

bars

alt.Chart(...)

#### Chart 5 : Yearly ESB Connections of Carlow County Council for any Month (user to input month value).

In [14]:
chart5 = df.copy()
chart5 = untidy_df(chart5)

options = {1 : 'Jan',
           2 : 'Feb',
           3 : 'Mar',
           4 : 'Apr',
           5 : 'May',
           6 : 'Jun',
           7 : 'Jul',
           8 : 'Aug',
           9 : 'Sept',
           10: 'Oct',
           11: 'Nov',
           12: 'Dec'
          }

In [15]:
for i,j in options.items():
    print (str(i) +' : ' + j)
    
user_inp = input('Enter a month number : ')

1 : Jan
2 : Feb
3 : Mar
4 : Apr
5 : May
6 : Jun
7 : Jul
8 : Aug
9 : Sept
10 : Oct
11 : Nov
12 : Dec
Enter a month number : 3


In [16]:
chart = alt.Chart(chart5)

bar = chart.mark_area(
    color = "lightblue",
    interpolate = 'step-after',
    line = True
).encode(
    x = 'Year:N',
    y = options[int(user_inp)]
).transform_filter(alt.datum['Council_Name'] == 'Carlow'
).properties(
    width = 500,
    title = f"Yearly ESB Connections of Carlow County Council, {options[int(user_inp)]}",
    background = "white",)

bar.save('C00246376_Python_Assignment_3_Part_3/static/chart5.html')

bar

alt.Chart(...)